In [8]:
from google.colab import files
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# 파일 업로드 창 열기 및 첫 번째 파일명 가져오기
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# 1. CSV 파일 불러오기 및 전처리
df = pd.read_csv(file_name, encoding='cp949', skiprows=9)
df.columns = ['번호', '발명의명칭', '요약', '청구항']

# NA 제거 및 텍스트 결합
df.dropna(subset=['발명의명칭', '요약', '청구항'], inplace=True)
df['all_text'] = df['발명의명칭'] + " " + df['요약'] + " " + df['청구항']

# 2. 텍스트 전처리
def preprocess(text):
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text

df['clean_text'] = df['all_text'].apply(preprocess)

# 3. DTM 생성 & LDA 학습
vectorizer = CountVectorizer(stop_words='english', max_df=0.95, min_df=2)
dtm = vectorizer.fit_transform(df['clean_text'])

k = 4
lda_model = LatentDirichletAllocation(n_components=k, random_state=1234)
lda_model.fit(dtm)

# 4. 토픽별 상위 단어 추출
terms = vectorizer.get_feature_names_out()
top_words = []
for topic_idx, topic in enumerate(lda_model.components_):
    top_features = [terms[i] for i in topic.argsort()[:-11:-1]]
    top_words.append(top_features)

# 5. 각 토픽의 대표 문서 10개 추출
topic_distribution = lda_model.transform(dtm)
topic_top_docs = {}

for topic_idx in range(k):
    top_doc_indices = topic_distribution[:, topic_idx].argsort()[::-1][:10]
    topic_top_docs[f"Topic_{topic_idx + 1}"] = df.iloc[top_doc_indices][['번호', '발명의명칭']].reset_index(drop=True)

# 6. 결과 출력
print("📌 각 토픽별 주요 단어:")
for i, words in enumerate(top_words, start=1):
    print(f"\n🔷 Topic {i}:")
    for w in words:
        print(f" - {w}")

print("\n📁 각 토픽별 대표 문서:")
for topic, docs in topic_top_docs.items():
    print(f"\n🔶 {topic} 대표 문서:")
    print(docs.to_string(index=False))


Saving 자율 운항 선박.csv to 자율 운항 선박.csv
📌 각 토픽별 주요 단어:

🔷 Topic 1:
 - 선박의
 - 선박
 - 있어서
 - 단계
 - 하는
 - 시스템
 - 정보를
 - 운항
 - 위한
 - 항에

🔷 Topic 2:
 - 선박
 - 정보를
 - 정보
 - 있어서
 - 단계
 - 항에
 - 타겟
 - 방법
 - 선박의
 - 이용하여

🔷 Topic 3:
 - 있어서
 - 제어
 - 항에
 - 하는
 - 선박
 - 시스템
 - 것을
 - 포함하는
 - 특징으로
 - 또는

🔷 Topic 4:
 - 있어서
 - 선박
 - 하는
 - 것을
 - 특징으로
 - 항에
 - 또는
 - 선박의
 - 포함하는
 - 위한

📁 각 토픽별 대표 문서:

🔶 Topic_1 대표 문서:
 번호                                                                                발명의명칭
352                                 진단, 안전진단, 시설진단, 원격진단, IT진단, AI진단, IoT진단의 장비, 장치, 시스템
356                    로봇, 건설로봇, 공사로봇, 작업로봇, 배달로봇, 서비스로봇, 도장로봇, 코팅로봇, 분사로봇, 청소로봇, 숏크리트로봇
485                                                    수중 음향 네트워크의 정밀 시간-경계 시분할 다중 접근 방법
500                                                       수중 이동체의 위치 추적을 위한 수중 통합 항법 시스템
327                                                                 선박 네트워크 접근제어 방법 및 장치
489                                                               관제 방법,